In [1]:
from astropy.io import fits

from astropy.table import Table

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from astropy.table import Table, join

In [2]:
#Reading in data file

fits_NSA_file = r'C:\Users\caeho\OneDrive\Documents\University of Rochester REU\NSA_v1_0_1_VAGC_vflag-V2-VF.fits'
#data = r'C:\Users\caeho\OneDrive\Documents\University of Rochester REU\NSA_v1_0_1_VAGC_vflag-V2-VF.fits'

hdul = fits.open(fits_NSA_file) 

In [3]:
# Names of all the collumns in the data base

data_header = hdul[1].header
data_header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 4091 / length of dimension 1                          
NAXIS2  =               641409 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                  141 / number of table fields                         
TTYPE1  = 'IAUNAME '                                                            
TFORM1  = '19A     '                                                            
TTYPE2  = 'SUBDIR  '                                                            
TFORM2  = '27A     '                                                            
TTYPE3  = 'RA      '        

In [4]:
data = hdul[1].data

In [5]:
data_table = Table()

data_table['aimc'] = data['aimc']
data_table['IN_DR7_LSS'] = data['IN_DR7_LSS']
data_table['RA'] = data['RA']
data_table['DEC'] = data['DEC']
data_table['Z'] = data['Z']
data_table['ELPETRO_ABSMAG_r'] = data['ELPETRO_ABSMAG'][:,4]
data_table['u_r'] = data['u_r']
data_table['sSFR'] = data['sSFR']
data_table['HImass'] = data['HImass']
data_table['flag3727'] = data['flag3727']
data_table['t3'] = data['t3']
data_table['Z12logOH'] = data['Z12logOH']
data_table['Zerr'] = data['Zerr']
data_table['logNO'] = data['logNO']
data_table['NOerr'] = data['NOerr']
data_table['vflag_VF'] = data['vFlag_VF']
data_table['ID#'] = [i for i in range(len(data_table))]
data_table['BPTclass'] = data['BPTclass']
data_table['ELPETRO_MASS'] = data['ELPETRO_MASS'] #using for the change in sSFR calcuation for metallicity

## Import Emission Line Ratios


In [6]:
#line_ratios = Table.read('line_ratios_test2.txt', format = 'ascii.commented_header')
line_ratios = Table.read('line_ratios.txt', format = 'ascii.commented_header')
line_ratios[:5]

ID,N2,O3N2,N2O2,N2_err,O3N2_err,N2O2_err
int32,float64,float64,float64,float64,float64,float64
0,0.4947436433131245,0.001255013649526481,nan,0.03437708322122293,0.0002788940323672723,0.09309809440111833
1,0.5657208512869216,0.006401026006795794,0.5264435955325992,0.060459862987763804,0.0018919426106776987,0.08748463119953064
2,0.20637675851271112,0.021678865176341867,0.2229008133253036,0.027143914863317953,0.004412781747401044,0.04343572346945603
3,0.17226625792173916,0.17855286821060004,0.01223838468977084,6.872230375586703,6.598236507502017,9.21386599340642
4,0.31831424677839676,0.003361059935014224,0.6232174068529841,0.03204577655418447,0.00100422186012563,0.09787017634556416


### Adding line ratios to the full data table

In [7]:
data_table = join(data_table, line_ratios, keys_left='ID#', keys_right='ID', join_type='left')

In [8]:
data_table[:5]

aimc,IN_DR7_LSS,RA,DEC,Z,ELPETRO_ABSMAG_r,u_r,sSFR,HImass,flag3727,t3,Z12logOH,Zerr,logNO,NOerr,vflag_VF,ID#,BPTclass,ELPETRO_MASS,ID,N2,O3N2,N2O2,N2_err,O3N2_err,N2O2_err
float64,float64,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int32,float64,float32,int32,float64,float64,float64,float64,float64,float64
-99.0,0.0,146.714215072841,-1.0412800233313741,0.021222278,-19.303656,2.3124398291110992,nan,nan,nan,nan,nan,nan,nan,nan,9,0,nan,6833158000.0,0,0.4947436433131245,0.001255013649526481,nan,0.03437708322122293,0.0002788940323672723,0.09309809440111833
1.0,1.0,146.6285851269414,-0.7651620976591762,0.06465632,-19.976496,2.976039946079254,-11.241625785827637,nan,nan,nan,nan,nan,nan,nan,2,1,3.0,12135996000.0,1,0.5657208512869216,0.006401026006795794,0.5264435955325992,0.060459862987763804,0.0018919426106776987,0.08748463119953064
-99.0,0.0,146.63170644325592,-0.9883467023052999,0.05265425,-18.431812,1.7119742929935455,nan,nan,nan,nan,nan,nan,nan,nan,2,2,nan,1788938800.0,2,0.20637675851271112,0.021678865176341867,0.2229008133253036,0.027143914863317953,0.004412781747401044,0.04343572346945603
1.0,1.0,146.9341288729797,-0.6704053610430766,0.121270485,-21.559162,3.0934733152389526,-11.71275806427002,nan,nan,nan,nan,nan,nan,nan,9,3,-1.0,57484075000.0,3,0.17226625792173916,0.17855286821060004,0.01223838468977084,6.872230375586703,6.598236507502017,9.21386599340642
-99.0,1.0,146.9634997327016,-0.5447758299302171,0.055980586,-19.100992,2.044341266155243,nan,nan,nan,nan,nan,nan,nan,nan,2,4,nan,4724134000.0,4,0.31831424677839676,0.003361059935014224,0.6232174068529841,0.03204577655418447,0.00100422186012563,0.09787017634556416


In [9]:
mask = data_table['Z']>0     #removing objects that are not galaxies
data_table = data_table[mask]

In [10]:
mask = data_table['ELPETRO_ABSMAG_r']>-25 #Removing outlier galaxies with M < -25
data_table = data_table[mask]

# Calcuating New Metallicities (N2 Method, O3N2 Method, and N2O2 Method)


In [11]:
data_table['log_M_sSFR'] = 283.728 - 116.265 * np.log10(data_table['ELPETRO_MASS']) + 17.4403 * (np.log10(data_table['ELPETRO_MASS']))**2 - 1.17146 * (np.log10(data_table['ELPETRO_MASS']))**3 + 0.0296526 *(np.log10(data_table['ELPETRO_MASS']))**4


data_table['change_log_sSFR'] = data_table['sSFR'] - data_table['log_M_sSFR'] #have to take the absolute value of sSFR?

data_table['Z12logOH_N2'] = 9.12 + 0.58 * np.log10(data_table['N2']) - 0.19 * data_table['change_log_sSFR'] 
data_table['Z12logOH_O3N2'] = 8.98 - 0.32 * np.log10(data_table['O3N2']) - 0.18 * data_table['change_log_sSFR']   
data_table['Z12logOH_N2O2'] = 9.20 + 0.54 * np.log10(data_table['N2O2']) - 0.36 * data_table['change_log_sSFR']

C:\Users\caeho\AppData\Local\Temp\ipykernel_24080\4064565771.py:1: RuntimeWarning: divide by zero encountered in log10
  data_table['log_M_sSFR'] = 283.728 - 116.265 * np.log10(data_table['ELPETRO_MASS']) + 17.4403 * (np.log10(data_table['ELPETRO_MASS']))**2 - 1.17146 * (np.log10(data_table['ELPETRO_MASS']))**3 + 0.0296526 *(np.log10(data_table['ELPETRO_MASS']))**4
C:\Users\caeho\AppData\Local\Temp\ipykernel_24080\4064565771.py:6: RuntimeWarning: divide by zero encountered in log10
  data_table['Z12logOH_N2'] = 9.12 + 0.58 * np.log10(data_table['N2']) - 0.19 * data_table['change_log_sSFR']
C:\Users\caeho\AppData\Local\Temp\ipykernel_24080\4064565771.py:7: RuntimeWarning: divide by zero encountered in log10
  data_table['Z12logOH_O3N2'] = 8.98 - 0.32 * np.log10(data_table['O3N2']) - 0.18 * data_table['change_log_sSFR']
C:\Users\caeho\AppData\Local\Temp\ipykernel_24080\4064565771.py:8: RuntimeWarning: divide by zero encountered in log10
  data_table['Z12logOH_N2O2'] = 9.20 + 0.54 * np.lo

In [12]:
data_table[:20]

aimc,IN_DR7_LSS,RA,DEC,Z,ELPETRO_ABSMAG_r,u_r,sSFR,HImass,flag3727,t3,Z12logOH,Zerr,logNO,NOerr,vflag_VF,ID#,BPTclass,ELPETRO_MASS,ID,N2,O3N2,N2O2,N2_err,O3N2_err,N2O2_err,log_M_sSFR,change_log_sSFR,Z12logOH_N2,Z12logOH_O3N2,Z12logOH_N2O2
float64,float64,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int32,float64,float32,int32,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float64
-99.0,0.0,146.714215072841,-1.0412800233313741,0.021222278,-19.303656,2.3124398291110992,nan,nan,nan,nan,nan,nan,nan,nan,9,0,nan,6833158000.0,0,0.4947436433131245,0.001255013649526481,nan,0.03437708322122293,0.0002788940323672723,0.09309809440111833,-9.776093,nan,nan,nan,nan
1.0,1.0,146.6285851269414,-0.7651620976591762,0.06465632,-19.976496,2.976039946079254,-11.241625785827637,nan,nan,nan,nan,nan,nan,nan,2,1,3.0,12135996000.0,1,0.5657208512869216,0.006401026006795794,0.5264435955325992,0.060459862987763804,0.0018919426106776987,0.08748463119953064,-9.849518,-1.3921079635620117,9.24100978108516,9.932579564097487,9.550688864062717
-99.0,0.0,146.63170644325592,-0.9883467023052999,0.05265425,-18.431812,1.7119742929935455,nan,nan,nan,nan,nan,nan,nan,nan,2,2,nan,1788938800.0,2,0.20637675851271112,0.021678865176341867,0.2229008133253036,0.027143914863317953,0.004412781747401044,0.04343572346945603,-9.559387,nan,nan,nan,nan
1.0,1.0,146.9341288729797,-0.6704053610430766,0.121270485,-21.559162,3.0934733152389526,-11.71275806427002,nan,nan,nan,nan,nan,nan,nan,9,3,-1.0,57484075000.0,3,0.17226625792173916,0.17855286821060004,0.01223838468977084,6.872230375586703,6.598236507502017,9.21386599340642,-9.976532,-1.7362260818481445,9.006879083032793,9.531955308832202,8.792412403614268
-99.0,1.0,146.9634997327016,-0.5447758299302171,0.055980586,-19.100992,2.044341266155243,nan,nan,nan,nan,nan,nan,nan,nan,2,4,nan,4724134000.0,4,0.31831424677839676,0.003361059935014224,0.6232174068529841,0.03204577655418447,0.00100422186012563,0.09787017634556416,-9.723511,nan,nan,nan,nan
2.0,1.0,146.96352370262414,-0.7593404235056136,0.09708638,-20.089943,2.836669385433197,-10.860784530639648,nan,nan,nan,nan,nan,nan,nan,2,5,5.0,12489805000.0,5,0.7284012546697451,0.194473268150988,0.20054884154799274,0.18028618700163646,1.2663686821427482,0.12000276651775059,-9.852844,-1.0079402923583984,9.231683652961154,9.388994080579156,9.186057390386843
-1.0,1.0,146.85984630722874,-0.8089065016506019,0.12658988,-21.062517,2.6804080605506897,-12.071269035339355,nan,nan,nan,nan,nan,nan,nan,9,6,-1.0,38875206000.0,6,2.4177401353874397,0.03606918174968211,0.3038391817959875,1.1447232694885159,0.18983204402506162,0.5857125392504999,-9.957764,-2.1135053634643555,9.74394359865935,9.822147352557094,9.681489570907425
40.35,1.0,146.5927838471015,-0.7602574022145185,0.064959235,-19.976143,2.08747461438179,-10.100362777709961,nan,1.0,6.100532417853275,6.363146339155186,0.33988585308459446,-0.18586781042085843,0.404887347385603,2,7,1.0,8501121500.0,7,0.3697980730155581,0.002610767946147577,0.6175497954293531,0.022621542798595125,0.0006549799389314251,0.09667950005956748,-9.805298,-0.29506492614746094,8.925481829348604,9.859745839672499,9.193186243794619
-2.0,1.0,146.72838067287842,-0.5569743913807053,0.089311965,-19.900196,2.5430691242218018,-11.400724411010742,nan,nan,nan,nan,nan,nan,nan,2,8,2.0,9400361000.0,8,0.24223345900307594,0.02426522164951037,0.04789182058309896,0.30414997476158334,0.18934957004947725,0.06480939050511576,-9.818176,-1.5825481414794922,9.063539942735755,9.781663701398578,9.057058458225855


In [13]:
data_table.write('new_metallicities.txt', format = 'ascii.commented_header', overwrite = True,
                 include_names = ['ID', 'Z12logOH_N2','Z12logOH_N2O2', 'Z12logOH_O3N2'])

## New table with only the ID and new metallicities

In [14]:
#new_metallicities = Table()

In [15]:
'''new_metallicities['ID'] = data_table['ID']
new_metallicities['Z12logOH_N2'] = data_table['Z12logOH_N2']
new_metallicities['Z12logOH_O3N2'] = data_table['Z12logOH_O3N2']
new_metallicities['Z12logOH_N2O2'] = data_table['Z12logOH_N2O2']

new_metallicities''';

In [16]:
#new_metallicities[:20]

In [17]:
#new_metallicities.write('new_metallicities2.txt', format = 'ascii.commented_header', overwrite = True)